In [91]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from keras.layers.convolutional import Convolution2D
from keras.callbacks import ModelCheckpoint
import tqdm
import pandas as pd
import os
import cv2

In [92]:
Center_image = 0
Left_image = 1
Right_iamge = 2
Steering_angle = 3
INPUT_SHAPE = (64, 64, 3)
learning_rate = 0.001
BATCH_SIZE = 32
EPOCHS = 50

In [93]:
def loadImage(path = "../data/driving_log.csv"):
    '''
    this fun is used for load the traning data from the tranning image folder and rade the csv file
    output is the trining files name and according steering value.
    '''
    # loand the csv file
    driving_data = pd.read_csv(path,header=None)
    #print(driving_data.describe())
    
    # assign x and y
    
    X = driving_data[[Center_image, Left_image, Right_iamge]].values
    y = driving_data[Steering_angle].values
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)
    
    return X_train, X_valid, y_train, y_valid

In [94]:
def argument(image, angle,top = 60, bottom = 25,resize = INPUT_SHAPE):
    image = crop(image, top, bottom)
    image, angle = crop(image, angle)
    image = scipy.misc.imresize(image, resize)
    
    
    
    

In [95]:
def crop(image, top = 60, bottom = 25):
    # Crop the image, remove the top sky and the front of the care
    return image[top:-bottom, :, :]

In [96]:
def flip(image, angle, prob=0.5):
    coin = bernoulli.rvs(prob)
    if coin: 
        return np.fliplr(image), -angle
    return image, angle

In [97]:
def generator(X,y, batch_size=32, angle_argument = 0.22):
    
    num_samples = len(X)
    while 1: # Loop forever so the generator never terminates
        batch_i = 0
        images = []
        angles = []
        
        for i in np.random.permutation(X.shape[0]):
            Center, Left, Right = X[i]
            angle = y[i]
            
            random_img = np.random.randint(0, 3)
            if random_img == Center_image:
                name = './IMG/'+Center.split('/')
                angle = float(angle)
            elif random_img == Left_image:
                name = './IMG/'+Left.split('/')
                angle = float(angle) + angle_argument
            elif random_img == Right_iamge:
                name = './IMG/'+Right.split('/')
                angle = float(angle) - angle_argument
            
            image = cv2.imread(name)
            image = argument(image, angle)
            
            images.append(image)
            angles.append(angle)

        # trim image to only see section with road
        X_train = np.array(images)
        y_train = np.array(angles)
        yield X_train, y_train

In [98]:
def model():
    model = Sequential()
    model.add(Lambda(lambda x: x / 127.5 - 1.0, input_shape=INPUT_SHAPE))
    
    model.add(Convolution2D(24, 5, 5,activation='relu', border_mode='same',subsample=(2, 2),name='conv1'))
    model.add(Convolution2D(36, 5, 5,activation='relu', border_mode='same',subsample=(2, 2),name='conv2'))
    model.add(Convolution2D(48, 5, 5,activation='relu', border_mode='same',subsample=(2, 2),name='conv3'))
    model.add(Convolution2D(64, 3, 3,activation='relu', border_mode='same',subsample=(1, 1),name='conv4'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1),name='MaxPooling1'))
    model.add(Convolution2D(64, 3, 3,activation='relu', border_mode='same',subsample=(1, 1),name='conv5'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1),name='MaxPooling2'))
    
    model.add(Flatten(name='flatten'))
    model.add(Dense(1164, activation='relu',name='Dense1'))
    model.add(Dense(100 , activation='relu',name='Dense2'))
    model.add(Dense(50  , activation='relu',name='Dense3'))
    model.add(Dense(10  , activation='relu',name='Dense4'))
    model.add(Dense(1   , activation='relu',name='Dense5'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=learning_rate),metrics=['accuracy'])
    return model


In [99]:
def train(model,X_train, X_valid, y_train, y_valid):
    model.fit_generator(generator(X_train,y_train, batch_size = BATCH_SIZE, angle_argument = 0.22),
                        samples_per_epoch = X_train.shape[0] // BATCH_SIZE,
                        nb_epoch=EPOCHS,
                        validation_data = generator(X_valid,y_valid, batch_size = BATCH_SIZE, angle_argument = 0.22),
                        nb_val_samples = len(X_valid),
                        callbacks=[ModelCheckpoint('tranning.model', monitor='val_acc', save_best_only=True)],
                        verbose=1)
    

In [100]:
X_train, X_valid, y_train, y_valid = loadImage()
model =model()
#model.summary()
train(model,X_train, X_valid, y_train, y_valid)
print("Liang Xu")


Epoch 1/50


Exception in thread Thread-7:
Traceback (most recent call last):
  File "/home/liangxu/anaconda2/envs/carnd-term1/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/home/liangxu/anaconda2/envs/carnd-term1/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/home/liangxu/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py", line 429, in data_generator_task
    generator_output = next(self._generator)
  File "<ipython-input-97-0563eaeae493>", line 18, in generator
    name = './IMG/'+Left.split('/')
TypeError: Can't convert 'list' object to str implicitly



ValueError: output of generator should be a tuple (x, y, sample_weight) or (x, y). Found: None